<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.6%20Modelaci%C3%B3n%20supervisada%20y%20no%20supervisada%20de%20Big%20Data/4.6.6%20Ensambles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ensambles

En este notebook se probarán 3 algoritmos diferentes basados en árboles:

* Árbol de decisión (Decision Tree)
* Bosques aleatorios (Random Forest)
* Árboles potenciados por gradiente (Gradient Boosting)

---

#### Conjunto de Datos

Usaremos un conjunto de datos del sector educación para clasificar a las universidades como privadas o públicas en función de estas características:

* School: Nombre de la institución
* Private: Un factor con niveles No y Sí que indican universidad pública o privada
* Apps: Número de solicitudes recibidas
* Accept: Número de solicitudes aceptadas
* Enroll: Número de nuevos estudiantes matriculados
* Top10perc: Porcentaje de nuevos estudiantes del top 10% de las clases de preparatoria
* Top25perc: Porcentaje de nuevos estudiantes del top 25% de las clases de preparatoria
* F_Undergrad: Número de estudiantes universitarios de tiempo completo
* P_Undergrad: Número de estudiantes universitarios a tiempo parcial
* Outstate: Matrícula fuera del estado
* Room_Board: Costos de alojamiento y comida
* Books: Costos estimados de libros
* Personal: Gastos personales estimados
* PhD: Porcentaje de profesores con doctorado
* Terminal: Porcentaje de profesores con título terminal (Terminal degree)
* S.F.Ratio: Proporción estudiantes/profesores
* Perc_alumni: Porcentaje de ex alumnos que donan
* Expend: Gastos por alumno (Incluye salarios y beneficios para maestros, servicios y gastos relacionados con actividades extracurriculares).
* Grad_Rate: Tasa de graduación

In [ ]:
generation = "mod4gen<x>"

In [ ]:
data = spark.read.csv(f"/mnt/{generation}/input/college.csv", inferSchema=True, header=True)

In [ ]:
data.printSchema()

In [ ]:
data.count(), len(data.columns)

In [ ]:
data.display()

In [ ]:
data.show(n=5)

In [ ]:
data.show(n=5, truncate=False)

### EDA

In [ ]:
data.groupBy("Private").count().display()

In [ ]:
data.groupBy("Private").count().display()

### Preprocesamiento

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
cols = data.columns[2:]
cols

In [ ]:
assembler = VectorAssembler(inputCols=cols, outputCol="features")

In [ ]:
type(assembler)

In [ ]:
output = assembler.transform(data)

In [ ]:
output.display()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")

In [ ]:
type(indexer)

In [ ]:
output_fixed = indexer.fit(output).transform(output)

In [ ]:
output_fixed.display()

In [ ]:
final_data = output_fixed.select("features", "PrivateIndex")

In [ ]:
final_data.display()

In [ ]:
train_data, val_data = final_data.randomSplit([0.7,0.3])

### Modelación

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [ ]:
help(RandomForestClassifier)

In [ ]:
# Se utilizan los hiperparámetros predeterminados para que la comparación sea "justa"

dtc = DecisionTreeClassifier(labelCol="PrivateIndex", featuresCol="features")
rfc = RandomForestClassifier(labelCol="PrivateIndex", featuresCol="features")
gbt = GBTClassifier(labelCol="PrivateIndex", featuresCol="features")

In [ ]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

### Importancia de variables

In [ ]:
dtc_model.featureImportances

In [ ]:
len(cols), len(rfc_model.featureImportances)

In [ ]:
list(zip(cols, gbt_model.featureImportances))

### Comparativa de modelos

In [ ]:
dtc_predictions = dtc_model.transform(val_data)
rfc_predictions = rfc_model.transform(val_data)
gbt_predictions = gbt_model.transform(val_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

[MulticlassClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.evaluation.MulticlassClassificationEvaluator.html)

In [ ]:
# Se evalúa el real contra el predicho calculando el error en la porción de prueba
f1_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="f1")
# acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")

In [ ]:
import numpy as np

In [ ]:
dtc_f1 = f1_evaluator.evaluate(dtc_predictions)
rfc_f1 = f1_evaluator.evaluate(rfc_predictions)
gbt_f1 = f1_evaluator.evaluate(gbt_predictions)

In [ ]:
print(f"F1-score: Decisión Tree: {np.round(dtc_f1, 2)}")
print(f"F1-score: Random Forest: {np.round(rfc_f1, 2)}")
print(f"F1-score: Grad Boosting: {np.round(gbt_f1, 2)}")